In [1]:
%load_ext autoreload
%autoreload 2

from model import *

from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

In [2]:
noisy_imgs_1, noisy_imgs_2 = torch.load('../../data/train_data.pkl')
train_input, train_target = noisy_imgs_1.float()/255.0, noisy_imgs_2.float()/255.0

In [3]:
input = train_input[0].view(1, 3, 32, 32)
target = train_target[0].view(1, 3, 32, 32)
input.shape

torch.Size([1, 3, 32, 32])

In [18]:
x = torch.randn((1, 3, 4, 4))

# x = torch.tensor(
    # [[0, 1., 0],     
    # [2, 0, 0],
    # [0, 0, 1]])

# y = torch.ones(x.shape)
y = torch.ones((1, 4, 3, 3))

x, y

(tensor([[[[-0.2572,  1.1405,  0.4362,  0.0368],
           [ 0.4210,  3.1108,  1.7982,  1.1641],
           [ 0.4797,  0.2791, -0.8702, -0.7707],
           [-2.2168,  0.5975,  0.2312,  0.8399]],
 
          [[-0.8350,  1.0374, -1.0337,  0.5612],
           [ 1.6940, -0.5218,  1.2199,  0.8497],
           [-0.9287, -1.5833, -0.4214,  0.4484],
           [ 0.7762,  0.1785, -0.1096, -0.7125]],
 
          [[-0.2799,  0.7910, -2.4349, -0.2604],
           [ 0.1306,  0.3162,  0.0051, -0.6362],
           [-1.6520, -0.0720,  0.6630, -0.3433],
           [ 0.4316,  0.0431,  0.0418, -1.4425]]]]),
 tensor([[[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]]]))

In [23]:
conv = nn.Conv2d(3, 4, 2)
expected = conv(x)
expected.shape

torch.Size([1, 4, 3, 3])

In [32]:
# torch.nn.Unfold(kernel_size=2)(x)

kernel_size = (2, 2)
out_channels = 4

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(x, kernel_size=kernel_size)
# unfolded

wxb = conv.weight.view(out_channels, -1) @ unfolded + conv.bias.view(1, -1, 1)
actual = wxb.view(1, out_channels, x.shape[2] - kernel_size[0] + 1, x.shape[3] - kernel_size[1]+ 1)
torch.testing.assert_allclose(actual, expected)

In [33]:
(conv.weight.view(out_channels, -1) @ unfolded).shape

torch.Size([1, 4, 9])

In [29]:
unfolded.shape

torch.Size([1, 12, 9])

### Testing our Conv2d Layer 

Forward pass

In [8]:
input.shape

torch.Size([1, 3, 32, 32])

In [10]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
x2 = my_conv.forward(input)
x2.shape

torch.Size([1, 4, 17, 17])

In [51]:
torch.testing.assert_allclose(my_conv.forward(input), torch.nn.functional.conv2d(input, my_conv.weight, my_conv.bias))

In [63]:

criterion = MSE()

loss = criterion.forward(x2, my_conv.forward(target))
loss.requires_grad_()
l_grad = loss.backward()
l_grad.shape

AttributeError: 'NoneType' object has no attribute 'shape'

Backward pass

In [21]:
x2.shape

torch.Size([1, 4, 3, 3])

In [22]:
(x2.view(3, -1) @ w.view(4, -1).t()).shape

torch.Size([4, 3, 2, 2])

In [23]:
my_conv.backward(x2)

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [4, 9] but got: [4, 3].

### Testing our TransposeConv2d layer

Forward pass

In [16]:
my_t_conv = TransposeConv2d(3, 5, 2, padding=1)
y_expected = my_t_conv.forward(target)
y_expected.shape

torch.Size([1, 5, 31, 31])

In [12]:
torch.testing.assert_allclose(my_t_conv.forward(x), torch.nn.functional.conv_transpose2d(x, my_t_conv.weight, my_t_conv.bias))

Backward pass

### Sequential model testing

In [36]:
x.shape

torch.Size([1, 3, 4, 4])

In [40]:
model = Sequential()
forward_pass = model.forward(x)

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])